# 0. Data Load

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [2]:
!pwd

/kaggle/working


In [3]:
from subprocess import check_output
import subprocess

files = check_output(['ls','../input/the-movies-dataset']).decode('utf8')
# files
for file in files.split("\n"):
#     print(file)
    path='../input/the-movies-dataset/'+ file
    popenobj=subprocess.Popen(['wc', '-l', path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     print(popenobj)
    result,error= popenobj.communicate()
    print("The file :",file,"has :",result.strip().split()[0],"rows")

The file : credits.csv has : b'45477' rows
The file : keywords.csv has : b'46420' rows
The file : links.csv has : b'45844' rows
The file : links_small.csv has : b'9126' rows
The file : movies_metadata.csv has : b'45572' rows
The file : ratings.csv has : b'26024290' rows
The file : ratings_small.csv has : b'100005' rows
The file :  has : b'0' rows


In [4]:
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings.csv')
links_small = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')
credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
movies_metadata = pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
ratings_small = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
links = pd.read_csv('/kaggle/input/the-movies-dataset/links.csv')

print('*** df 생성 완료 ***')

*** df 생성 완료 ***


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 1. Memory Optimization

### 1) ratings

In [5]:
# rationgs file memory check

mem_ratings = ratings.memory_usage(index=True).sum()
print("ratings dataset uses ", mem_ratings/ 1024**2," MB")
ratings.head()

ratings dataset uses  794.1983947753906  MB


,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [6]:
print(ratings.max())
print("--------------------")
print(ratings.min())
print("--------------------")
print(ratings.dtypes)

userId       2.708960e+05
movieId      1.762750e+05
rating       5.000000e+00
timestamp    1.501830e+09
dtype: float64
--------------------
userId               1.0
movieId              1.0
rating               0.5
timestamp    789652004.0
dtype: float64
--------------------
userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object


In [7]:
print(ratings.userId.max())
print(ratings.movieId.max())
print(ratings.timestamp.max())

270896
176275
1501829870


In [8]:
ratings['userId'] = ratings['userId'].astype(np.uint32)
ratings['movieId'] = ratings['movieId'].astype(np.uint32)
ratings['rating'] = ratings['rating'].astype(np.float16)
ratings['timestamp'] = ratings['timestamp'].astype(np.uint32)

#check memory
print(ratings.memory_usage(index=True))
new_mem_ratings=ratings.memory_usage(index=True).sum()
print("ratings dataset uses ",new_mem_ratings/ 1024**2," MB after changes")
print("memory saved =",(mem_ratings-new_mem_ratings)/ 1024**2," MB")

Index              128
userId       104097156
movieId      104097156
rating        52048578
timestamp    104097156
dtype: int64
ratings dataset uses  347.4618663787842  MB after changes
memory saved = 446.73652839660645  MB


### 2) credits 

In [9]:
mem_credits = credits.memory_usage(index=True).sum()
print("credits dataset uses ", mem_credits/ 1024**2," MB")
credits.head()

credits dataset uses  1.040985107421875  MB


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [17]:
print(credits.max())
print("--------------------")
print(credits.min())
print("--------------------")
print(credits.dtypes)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)